# Hipótese 2 
Neste Jupyter Notebook, exploraremos a hipótese de que, durante os meses de conscientização de infecções sexualmente transmitidas (Outubro e novembro), acontecem mais ativações do plano de saúde com o objetivo de conversar com urologistas e ginecologistas.

***
## Instalação das bibliotecas:
Instala bibliotecas necessárias:

In [ ]:
!pip install pandas numpy

***
## Importação das bibliotecas:
Importa bibliotecas necessárias:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

***
## Importe do banco de dados via arquivo local
Importação do DataFrame da base de dados da Unipar com a biblioteca pandas.<br> 
Para rodar o notebook corretamente, será necessário importar o arquivo de banco de dados manualmente, pois ele não está incluído no repositório devido à presença de dados sensíveis.

#### Passos para Importar o Banco de Dados no VS Code

4. *Obtenha o Arquivo de Dados*:
   - O arquivo BASE_DE_SINISTRO_UNIPAR_BRADESCO_ORIGINAL.csv deverá ser fornecido separadamente. Entre em contato com o responsável pelo projeto para receber o arquivo.

5. *Posicione o Arquivo na Pasta Correta*:
   - Após receber o arquivo, arraste-o para a mesma pasta onde o notebook está localizado em seu computador. Isso garante que o caminho relativo na função de leitura permaneça o mesmo e funcione corretamente.
   
6. *Verifique o Caminho do Arquivo*:
   - O código de leitura do arquivo já está implementado no notebook e não precisa ser alterado:
     python
     df = pd.read_csv('BASE_DE_SINISTRO_UNIPAR_BRADESCO_ORIGINAL.csv', decimal=',')
     
   - Certifique-se de que o arquivo CSV esteja no mesmo diretório que o notebook para evitar problemas de caminho.

7. *Rodar o Notebook*:
   - Com o arquivo posicionado corretamente, execute o notebook normalmente. O pandas irá carregar os dados e você poderá seguir com a análise.

*Nota*: Caso o arquivo não esteja na mesma pasta que o notebook, o código não será capaz de localizar o banco de dados, resultando em um erro. Portanto, é essencial que o arquivo seja arrastado para o diretório correto antes da execução.

Agora que você baixou os notebooks e o banco de dados, pode seguir para os próximos passos de execução, seja localmente ou no Google Colab.

In [3]:
df = pd.read_csv('[NÃO EDITAR] BASE DE SINISTRO UNIPAR_BRADESCO_ORIGINAL.xlsx - BASE.csv', decimal=',')

***
## Pré-Processamento
Como apresentado no notebook 'Pré-Processamento', as linhas a seguir executam várias etapas para limpar e organizar o banco de dados.<br>
Os comentários no código explicam cada ação.

In [4]:
# Tratamento de valores nulos
df = df.dropna()
# Correção do ponto faltante em 'UNIPAR INDUPA DO BRASIL S.A'
df = df.replace({"UNIPAR INDUPA DO BRASIL S.A": "UNIPAR INDUPA DO BRASIL S.A."})
# Remoção de AGREGADO e DEPENDENTE
df_remove_d = df.loc[(df['Elegibilidade Sinistro'] == 'DEPENDENTE') ]
df = df.drop(df_remove_d.index)
df_remove_a = df.loc[(df['Elegibilidade Sinistro'] == 'AGREGADO') ]
df = df.drop(df_remove_a.index)
# Tratamento de valores duplicados
df = df.drop_duplicates(keep='last')
# Convertendo a coluna 'Dt Data Sinistro' para o formato 'YYYYMMDD' (Ano-Mês-Dia)
df['Dt Data Sinistro'] = pd.to_datetime(df['Dt Data Sinistro'], format='%d/%m/%Y').dt.strftime('%Y%m%d')

In [5]:
# Converter a coluna para datetime
df['Dt Data Sinistro'] = pd.to_datetime(df['Dt Data Sinistro'], format='%Y%m%d')

# Extrair componentes de ano, mês e dia
df['Ano'] = df['Dt Data Sinistro'].dt.year
df['Mes'] = df['Dt Data Sinistro'].dt.month
df['Dia'] = df['Dt Data Sinistro'].dt.day

In [ ]:
df

***
## Comprovação da hipótese

O código abaixo apresenta um gráfico que mostra a distribuição das ativações de sinistros por mês, considerando os anos de 2023 e 2024. Essa análise será baseada na coluna 'Descrição Serviço Sinistro'.

In [ ]:
df['Mes_Ano'] = df['Mes'].astype(str) + '-' + df['Ano'].astype(str)
sinistros_por_mes = df.groupby('Mes')['Descricao Servico Sinistro'].count()

sinistros_por_mes = sinistros_por_mes.reset_index()

plt.figure(figsize=(10, 6))
plt.plot(sinistros_por_mes['Mes'], sinistros_por_mes['Descricao Servico Sinistro'], marker='o', linestyle='-', color='g')

plt.xticks(np.arange(1, 13, 1), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

plt.title('Número de ativações de sinistro por mês')
plt.xlabel('Mês')
plt.ylabel('Número de ativações')
plt.grid(True)

plt.show()

Esse gráfico mostra que outubro e novembro estão levemente abaixo da média quando se trata de ativações do sinistro, mesmo ambos sendo meses de conscientização. Para fazer uma separação mais específica, separemos palavras chaves relacionadas a câncer de mama e próstata dentro dos dados. Com essas palavras separadas, podemos ver quais meses tem o plano de saúde ativado mais vezes com casos somente relacionados com câncer de mama e próstata.

In [8]:
keywords = [
    'câncer', 'mama', 'próstata', 'neoplasia', 'ginecologista'
    'tumor mamário', 'tumor de próstata', 'urologista'
    'adenocarcinoma', 'carcinoma', 'mamografia', 
    'biópsia', 'prostatectomia', 'mastectomia', 
    'hormonioterapia', 'quimioterapia', 'radioterapia'
]

***
## Ativações fora dos meses de conscientização.

Aqui aplicamos um filtro para que o "df_total_doencas" mantenha apenas as linhas em que a coluna "Descrição Servico Sinistro" contém alguma das palavras presentes na variável "keywords".

In [9]:
df_total_procuras = df[df['Descricao Servico Sinistro'].str.contains('|'.join(keywords), case=False, na=False)]

Número total de linhas:

In [ ]:
print(f'Total de linhas: {df_total_procuras['Descricao Servico Sinistro'].shape[0]}')

Em seguida é plotado um gráfico que agruda os valores nos meses em que eles ocorreram.

In [ ]:
procuras_por_mes = df_total_procuras.groupby('Mes')['Descricao Servico Sinistro'].count().reset_index()

plt.figure(figsize=(10, 6))
plt.scatter(procuras_por_mes['Mes'], procuras_por_mes['Descricao Servico Sinistro'], color='b')

plt.plot(procuras_por_mes['Mes'], procuras_por_mes['Descricao Servico Sinistro'], linestyle='-', color='b')

plt.title('Dispersão das ativações relacionadas a câncer de mama ou próstata por mês')
plt.xlabel('Mês')
plt.ylabel('Número de ativações')
plt.xticks(np.arange(1, 13, 1), ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
plt.grid(True)
plt.savefig('ativacoes_procuras__por_mes.png', dpi=300, bbox_inches='tight')
plt.show()

Como podemos ver, o gráfico mostra que a ativação do plano de saúde relacionado ao câncer de mama ou próstata é mais frequente nos meses de fevereiro e junho. Isso contradiz nossa hipótese inicial, que diz que os meses com mais procura seriam outubro e novembro.

É importante saber que,depois de aplicar o filtro das keywords, o banco de dados desceu para somente 135 registros.

***
## Ativações durante os meses de conscientização

Para a etapa final da comprovação dessa hipótese, vamos filtrar de novo nossos dados para que "df_procuras" tenha somente os registros dos meses de outubro e novembro, os dois meses de conscientização.

In [ ]:
df_procuras = df.loc[((df['Ano'] == 2023) | (df['Ano'] == 2024)) & (df['Mes'] >= 10) & (df['Mes'] <= 11)]
df_procuras.shape[0]

Novamente vamos filtrar as palavras-chave.

In [ ]:
df_outnov = df_procuras[df_procuras['Descricao Servico Sinistro'].str.contains('|'.join(keywords), case=False, na=False)]
df_outnov['Descricao Servico Sinistro']

Número total de linhas

In [ ]:
print(f'Total de linhas: {df_outnov['Descricao Servico Sinistro'].shape[0]}')

Agora, vamos finalmente plotar os gráficos com as ativações em outubro e novembro.

In [ ]:
sinistros_ativacao_por_mes = df_outnov.groupby('Mes')['Descricao Servico Sinistro'].count().reset_index()
plt.figure(figsize=(10, 6))
plt.plot(sinistros_ativacao_por_mes['Mes'], sinistros_ativacao_por_mes['Descricao Servico Sinistro'], marker='o', linestyle='-', color='b')

plt.xticks(np.arange(10, 12), ['Out', 'Nov'])

plt.title('Número de ativações de sinistro nos meses de conscientização')
plt.xlabel('Mês')
plt.ylabel('Número de ativações')
plt.grid(True)
plt.show()

Após filtrar os dados pelas keywords novamente, incrívelmente sobraram só 22 registros.

***

## Conclusão

Após plotar todos os gráficos e analisar os dados presentes, concluimos que, ao longo dos anos 2023 e 2024, procuras ao ginecologista ou urologista somente aconteceram 135 vezes, e somente 22 delas acontecem nos meses de conscientização. A maior concentração de procuras ocorre em junho e fevereiro, chegando quase em 20 ativações por mês. Em conclusão, foi confirmado que nossa hipótese estava incorreta: a busca por ginecologistas ou urologistas não acontecem em maior concentração nos meses de conscientização.